<a href="https://colab.research.google.com/github/Yash-Kamtekar/Optional_Assignment/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First import the libraries which will be useful.

In [7]:
from collections import Counter
from itertools import combinations
import numpy as np

custom data set

In [8]:
transactions = {
    1: ["1", "3", "4"],
    2: ["2", "3", "5"],
    3: ["1", "2", "3", "5"],
    5: ["2", "5"],
    6: ["1", "3", "5"]
}

setting minimum support and confidence value.

In [9]:
min_support_count = 2
min_confidence_value = 0.6

Function to prune.

In [10]:
def _pruning(current, previous, size):
    final_keys = []
    previous = [tuple(i) for i in previous]
    for key in current:
        FLAG = False
        current_comb = list(combinations(key, size))
        for i in current_comb:
            if i in previous or i[::-1] in previous:
                FLAG = True
            else:
                FLAG = False
                break

        if FLAG:
            final_keys.append(key)

    return final_keys

function to calculate Support.

In [11]:
def support_value(itemset_keys_, transactions):
    itemset = {key: 0 for key in itemset_keys_}

    for keys in itemset_keys_:
        for val in transactions.values():
            if set(keys) & set(val) == set(keys):
                itemset[keys] += 1
    return itemset

creating frequent itemset.

In [12]:
def get_frequent_itemset(size=None, transactions=None, itemset=None):
    if size == 1:
        itemset = Counter()

        for val in transactions.values():
            itemset.update(val)
    else:
        prev_itemset_keys = list(itemset.keys())
        prev_itemset = itemset.copy()

        valid_keys = list(set(itemset.keys()))

        l = []
        for row in valid_keys:
            l.extend(row)

        valid_keys = set(l)

        # candidate itemset keys
        itemset_keys_ = list(combinations(valid_keys, size))

        # Apriori algorithm is based on the concept that a subset of a frequent itemset must also be a frequent itemset
        # so we are pruning away those features whose subset are not present in the previous frequent itemset
        if size >= 2:
            itemset_keys_ = _pruning(
                itemset_keys_, prev_itemset_keys, size - 1)

        # finding support value for each of the selected itemset feature combination
        itemset = support_value(itemset_keys_, transactions)

        # defaulting back to th previous frequent itemset if
        # the iteration doesn't find any itemset which has the theshold required
        if itemset == {}:
            itemset = prev_itemset

    # getting frequent itemset from itemset
    # Frequent Itemset is an itemset whose support
    # value is greater than a threshold value(support).

    frequent_itemset = {}
    for key, val in itemset.items():
        if val >= min_support_count:
            frequent_itemset[key] = val

    return frequent_itemset

function to find subset

In [13]:
def finding_subsets(frequent_set):
    item_list = []
    size = len(list(frequent_set.keys())[0])
    for key in frequent_set.keys():
        subsets = []
        for i in range(1, size):
            subsets.append(list(combinations(key, i)))

        subsets = list(np.array(subsets).flatten())
        subsets.insert(0, key)
        item_list.append(subsets)

    return item_list

function to find rules.

In [14]:
def finding_rules(itemset_sub):
    print("Antecedents -->  Consequents --- Confidence")
    for i in range(1, len(itemset_sub)):

        # passing as list as we have designed support_value function as
        # a function that takes an iteratable list of itemsets
        x = support_value([itemset_sub[0], ], transactions)
        y = support_value([itemset_sub[i], ], transactions)
        confidence = list(x.values())[0] / list(y.values())[0]
        if confidence >= min_confidence_value:
            print(
                f"{itemset_sub[i]} --> {itemset_sub[0]} --- {round(confidence, 2)}")

In [15]:
print("""
    ITEMS
    1: Banana
    2: Eggs
    3: Milk
    4: Tea
    5: Bread
""")

f = {}

for i in range(1, 5):
    f = get_frequent_itemset(size=i, transactions=transactions,
                             itemset=f)

# frequent_itemsets

print("Frequent Itemsets...")
for key, val in f.items():
    print(f"Itemset: {key}, support value: {val}")


subset = finding_subsets(f)

for i in subset:
    print(f"Rules for itemset - {i[0]}")
    finding_rules(i)
    print()


    ITEMS
    1: Banana
    2: Eggs
    3: Milk
    4: Tea
    5: Bread

Frequent Itemsets...
Itemset: ('1', '5', '3'), support value: 2
Itemset: ('2', '5', '3'), support value: 2
Rules for itemset - ('1', '5', '3')
Antecedents -->  Consequents --- Confidence
('1',) --> ('1', '5', '3') --- 0.67
('1', '5') --> ('1', '5', '3') --- 1.0
('1', '3') --> ('1', '5', '3') --- 0.67
('5', '3') --> ('1', '5', '3') --- 0.67

Rules for itemset - ('2', '5', '3')
Antecedents -->  Consequents --- Confidence
('2',) --> ('2', '5', '3') --- 0.67
('2', '5') --> ('2', '5', '3') --- 0.67
('2', '3') --> ('2', '5', '3') --- 1.0
('5', '3') --> ('2', '5', '3') --- 0.67



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
